In [110]:
#Read files

import pandas as pd
import glob
from os import walk
import re
import sys

def getDF(path):
    for (dirpath,dirnames,filenames) in walk(path):
        break
    #Populate dataframe
    df = pd.DataFrame(['lose lbs'], columns=['Content'])
    for fname in filenames:
        f = open(path+fname,'r')
        text = f.read()
        df_tmp = pd.DataFrame([text], columns=['Content'])
        df = df.append(df_tmp, ignore_index=True)
    return df

def splitData(spam, ham):
    spam_train = spam.sample(frac = 0.8, random_state=0)
    spam_test = spam.drop(spam_train.index)
    ham_train = ham.sample(frac = 0.8, random_state=0)
    ham_test = ham.drop(ham_train.index)
    return [spam_train, ham_train, spam_test, ham_test]

def gen2grams(txt):
    global curGram
    words = re.split('[^A-Za-z]+', txt.lower())
    words = filter(None, words)
    for i in xrange(len(words)-1):
        key = (words[i], words[i+1])
        if curGram.has_key(key):
            curGram[key] += 1
        else:
            curGram[key] = 1
    
def populate2Gram(data):
    global curGram
    curGram = dict()
    data['Content'].apply(gen2grams)
    return curGram

spam = getDF('spamemails/other/SPAM/')
ham = getDF('spamemails/other/HAM/')
(spam_train, ham_train, spam_test, ham_test) = splitData(spam, ham)

#Obtain n gram
spam_2gram = populate2Gram(spam_train)
ham_2gram = populate2Gram(ham_train)
total_2gram = populate2Gram(spam_train.append(ham_train))


In [97]:
def findSums(gram2):              
    words = set([element[0] for element in gram2])
    wlen = dict()
    for word in words:
        wlen[word] = sum([gram2[element] for element in gram2 if element[0] == word])
    return wlen

#Find sums
slen = findSums(spam_2gram)
hlen = findSums(ham_2gram)
tlen = findSums(total_2gram)

In [107]:
import math
def pairProb(word1, word2, gram2, length):
    if (word1,word2) in gram2:
        d = length[word1]
        n = float(gram2[word1,word2])
        return n/d
    else:
        return math.e ** -5
    
def lineProb(txt, gram2, length):
    #need to implement smoothing
    words = re.split('[^A-Za-z]+', txt.lower())
    words = filter(None, words)
    prob = 1.0
    for i in range(len(words)-1):
        prob = prob * pairProb(words[i],words[i+1], gram2, length)
        p = pairProb(words[i],words[i+1], gram2, length)
    return prob

In [112]:
def test(text):
    actualSpam = False
    #Bayes Theorem
    ES = lineProb(text,spam_2gram, slen)
    EH = lineProb(text,ham_2gram, hlen)
    S = H = 0.5
    E = lineProb(text,total_2gram, tlen)
    if E == 0:
        print 'Do not know'
        return
    SE = (ES * S) / E
    HE = (EH * H) / E
    prediction = SE > HE
    return prediction == actualSpam

#spam_test['Content'].apply(test) 
#ham_test['Content'].apply(test) 